## Crypto data downloader

In [1]:
starting_date = '2017-01-01T00:00:00Z'
ending_date = '2022-04-25T00:00:00Z'
pair = 'BTC/USDT'
tf = "1d"
exchange = 'Binance'

to_present = True
pair_id = pair.split('/')[0] + "-" + pair.split('/')[-1]
print("Pair : " + pair_id)

Pair : BTC-USDT


In [2]:
import ccxt
from datetime import datetime as dt
from freqtrade.data.converter import ohlcv_to_dataframe
import pandas as pd
import time

exchange_dict = {'Binance':ccxt.binance(),
                'Kucoin':ccxt.kucoin(),
                'Coinex':ccxt.coinex(),
                'Kraken':ccxt.kraken(),
                'Coinbase':ccxt.coinbase()}
ex = exchange_dict[exchange]

print(ex.iso8601(ex.milliseconds()), 'Loading markets')
#markets = ex.load_markets()
print(ex.iso8601(ex.milliseconds()), 'Markets loaded')

from_timestamp = ex.parse8601(starting_date)
if to_present:
    now = ex.milliseconds()
else:
    now = ex.parse8601(ending_date)
df_list = []
while from_timestamp < now:
    print('Fetching candles starting from', ex.iso8601(from_timestamp))
    try:
        ohlcvs = ex.fetch_ohlcv(pair, tf, from_timestamp)
    except:
        print("Retrying...")
        continue
    time.sleep(ex.rateLimit / 1000)
    if not len(ohlcvs):
        break
    from_timestamp = ohlcvs[-1][0] + ex.parse_timeframe(tf) * 1000
    df_list.append(ohlcv_to_dataframe(ohlcvs, tf, pair=pair, drop_incomplete=False))

2022-04-27T17:09:25.945Z Loading markets
2022-04-27T17:09:25.945Z Markets loaded
Fetching candles starting from 2017-01-01T00:00:00.000Z
Retrying...
Fetching candles starting from 2017-01-01T00:00:00.000Z
Retrying...
Fetching candles starting from 2017-01-01T00:00:00.000Z
Retrying...
Fetching candles starting from 2017-01-01T00:00:00.000Z
Fetching candles starting from 2018-12-30T00:00:00.000Z
Fetching candles starting from 2020-05-13T00:00:00.000Z
Fetching candles starting from 2021-09-25T00:00:00.000Z


In [22]:
df = pd.concat(df_list, axis=0)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
print('Starting date :', df['date'].iloc[0])
print('Ending date :', df['date'].iloc[-1])
print('DF is created.')

Starting date : 2017-08-17 00:00:00+00:00
Ending date : 2022-04-27 00:00:00+00:00
DF is created.


In [52]:
print('DF size: ', df.shape[0], 'rows')
df.head()

DF size:  1715 rows


,date,open,high,low,close,volume
0,2017-08-17 00:00:00+00:00,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18 00:00:00+00:00,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19 00:00:00+00:00,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20 00:00:00+00:00,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21 00:00:00+00:00,4069.13,4119.62,3911.79,4016.00,691.743060


In [55]:
df.columns = ["_id", "open", "high", "low", "close", "volume"]
df.head()

,_id,open,high,low,close,volume
0,2017-08-17 00:00:00+00:00,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18 00:00:00+00:00,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19 00:00:00+00:00,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20 00:00:00+00:00,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21 00:00:00+00:00,4069.13,4119.62,3911.79,4016.00,691.743060


In [76]:
df_dict = df.to_dict("records")
df_dict[-5:]

[{'_id': Timestamp('2022-04-23 00:00:00+0000', tz='UTC'),
  'open': 39709.19,
  'high': 39980.0,
  'low': 39285.0,
  'close': 39441.6,
  'volume': 20291.42375},
 {'_id': Timestamp('2022-04-24 00:00:00+0000', tz='UTC'),
  'open': 39441.61,
  'high': 39940.0,
  'low': 38929.62,
  'close': 39450.13,
  'volume': 26703.61186},
 {'_id': Timestamp('2022-04-25 00:00:00+0000', tz='UTC'),
  'open': 39450.12,
  'high': 40616.0,
  'low': 38200.0,
  'close': 40426.08,
  'volume': 63037.12784},
 {'_id': Timestamp('2022-04-26 00:00:00+0000', tz='UTC'),
  'open': 40426.08,
  'high': 40797.31,
  'low': 37702.26,
  'close': 38112.65,
  'volume': 66650.258},
 {'_id': Timestamp('2022-04-27 00:00:00+0000', tz='UTC'),
  'open': 38112.64,
  'high': 39474.72,
  'low': 37881.31,
  'close': 39145.71,
  'volume': 46360.89716}]

In [77]:
#Mongodb init:
# https://www.w3schools.com/python/python_mongodb_getstarted.asp
# https://www.mongodb.com/try/download/community
import pymongo

In [78]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["fin_db"]
mycol = mydb[pair]

x = mycol.insert_many(df_dict)

#print list of the _id values of the inserted documents:
#print(x.inserted_ids)

In [79]:
collist = mydb.list_collection_names()
print("collection list:", collist)
if pair in collist:
  print("The collection exists.")

collection list: ['BTC/USDT']
The collection exists.


In [80]:
# Find:
for x in mycol.find({"volume":46360.89716}):
  print(x)

{'_id': datetime.datetime(2022, 4, 27, 0, 0), 'open': 38112.64, 'high': 39474.72, 'low': 37881.31, 'close': 39145.71, 'volume': 46360.89716}


In [81]:
myquery = { "_id": dt(2022, 4, 27, 0, 0) }

mydoc = mycol.find(myquery)

for x in mydoc:
  print(x)

{'_id': datetime.datetime(2022, 4, 27, 0, 0), 'open': 38112.64, 'high': 39474.72, 'low': 37881.31, 'close': 39145.71, 'volume': 46360.89716}


In [82]:
for x in mycol.find():
   pass 
   #print(x)

In [75]:
# Delete a Table or collection:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["fin_db"]
mycol = mydb[pair]

mycol.drop()